This notebook prepares the data for the subsequent notebook `10-Step-Analyze-Chebyshev.ipynb`, which generates the figure illustrating the multi-step rollouts on non-uniform grids, as described in Supplementary Material Section 8.2.

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import torch
num_devices = torch.cuda.device_count()
print("Number of visible GPUs:", num_devices)

for i in range(num_devices):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

current_device = torch.cuda.current_device()
print("Current device index:", current_device)
print("Current device name:", torch.cuda.get_device_name(current_device))

import random
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

from tqdm import tqdm
from data_processing import (
    SimpleSerializerSettings, scale_2d_array, unscale_2d_array,
    serialize_2d_integers, deserialize_2d_integers
)

from allen_cahn_equation_Chebyshev import (
    compute_exact_solution_random_ic_vary_Nx,
    visualize_spline_ic,
    plot_both_grids
)

from llama_utils import load_model_and_tokenizer, llm_multi_predictions

MODEL_NAME = "meta-llama/Llama-3.1-8B"
# MODEL_NAME = "meta-llama/Llama-3.2-3B"
# MODEL_NAME = "meta-llama/Llama-3.2-1B"

# Set random seeds for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

In [ ]:
model, tokenizer = load_model_and_tokenizer(MODEL_NAME)

In [ ]:
# Example: Demonstrating the process of generating and visualizing a random initial condition
L = 2
Nx = 14
init_cond_random = np.random.uniform(-0.5, 0.5, size=Nx)
fig, cs = visualize_spline_ic(L, Nx, init_cond_random)
plt.tight_layout()
plt.show()

# Example: Demonstrating how to resample spatial points from an underlying random initial condition
Nx_original = Nx
Nx_new = 14
fig, cs, init_cond_random_new = plot_both_grids(L, Nx_original, Nx_new, init_cond_random)
plt.tight_layout()
plt.show()

In [ ]:
# Define parameters for the Allen-Cahn equation
L = 2       # Length of the spatial domain
k = 0.001   # Thermal diffusivity
T = 0.5     # Total simulation time
Nt = 25     # Number of time steps
dt = T/Nt
Nx = 14     # Number of spatial steps (excluding boundary points)
dx = L/(Nx+1)
settings = SimpleSerializerSettings(space_sep=",", time_sep=";")
input_time_steps = 16
number_of_future_predictions = 10
n_ics = 20      
n_runs_per_ic = 20

# Generate all random initial conditions and spline objects
stored_initial_conditions = []
stored_spline_objects = []
for ic_seed in range(n_ics):
    random.seed(ic_seed)
    np.random.seed(ic_seed)
    torch.manual_seed(ic_seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(ic_seed)
    init_cond_random = np.random.uniform(-0.5, 0.5, size=Nx)
    stored_initial_conditions.append(init_cond_random.copy())
    fig, cs = visualize_spline_ic(L, Nx, init_cond_random)
    plt.close(fig)
    stored_spline_objects.append(cs)

stored_initial_conditions_array = np.array(stored_initial_conditions)
all_llm_max_diffs = []
all_llm_rmses = []

for ic_seed in tqdm(range(n_ics)):
    # Use the stored initial condition and spline
    init_cond_random = stored_initial_conditions[ic_seed]
    cs = stored_spline_objects[ic_seed]
    # Compute exact solution for this initial condition on Chebyshev grid
    u_exact_cheb = compute_exact_solution_random_ic_vary_Nx(L, k, T, Nx, Nt, spline_obj=cs, use_chebyshev=True)
    u_exact_scaled_cheb, vmin_exact_cheb, vmax_exact_cheb = scale_2d_array(u_exact_cheb)
    u_exact_serialized_cheb = serialize_2d_integers(u_exact_scaled_cheb, settings)
    # Run LLM prediction for this initial condition
    llm_max_diffs_cheb, llm_rmses_cheb, _, _, _  = llm_multi_predictions(
        full_serialized_data=u_exact_serialized_cheb,
        input_time_steps=input_time_steps,
        number_of_future_predictions=number_of_future_predictions,
        model=model,
        tokenizer=tokenizer,
        Nx=Nx,
        settings=settings,
        vmin=vmin_exact_cheb,
        vmax=vmax_exact_cheb,
        n_seeds=n_runs_per_ic
    )

    all_llm_max_diffs.append(llm_max_diffs_cheb)
    all_llm_rmses.append(llm_rmses_cheb)

# Compute quant floor
all_baseline_max_errors_per_step = []
all_baseline_rmse_errors_per_step = []

for ic_seed in range(n_ics):
    # Use the stored initial condition and spline
    init_cond_random = stored_initial_conditions[ic_seed]
    cs = stored_spline_objects[ic_seed]
    # Compute exact solution for this initial condition on Chebyshev grid
    u_exact_cheb = compute_exact_solution_random_ic_vary_Nx(L, k, T, Nx, Nt, spline_obj=cs, use_chebyshev=True)
    # Quantization pipeline
    u_exact_scaled_cheb, vmin_exact_cheb, vmax_exact_cheb = scale_2d_array(u_exact_cheb)
    u_exact_serialized_cheb = serialize_2d_integers(u_exact_scaled_cheb, settings)
    u_exact_parsed_cheb = deserialize_2d_integers(u_exact_serialized_cheb, settings)
    u_exact_unscaled_cheb = unscale_2d_array(u_exact_parsed_cheb, vmin_exact_cheb, vmax_exact_cheb)
    # Compute baseline errors at each time step for this seed
    seed_max_errors_per_step = []
    seed_rmse_errors_per_step = []
    
    for t in range(u_exact_cheb.shape[0]):
            max_err_t  = np.max(np.abs(u_exact_cheb[t] - u_exact_unscaled_cheb[t]))
            rmse_err_t = np.sqrt(np.mean((u_exact_cheb[t] - u_exact_unscaled_cheb[t])**2))
            seed_max_errors_per_step.append(max_err_t)
            seed_rmse_errors_per_step.append(rmse_err_t)

    all_baseline_max_errors_per_step.append(seed_max_errors_per_step)
    all_baseline_rmse_errors_per_step.append(seed_rmse_errors_per_step)

all_baseline_max_errors_per_step  = np.array(all_baseline_max_errors_per_step)
all_baseline_rmse_errors_per_step = np.array(all_baseline_rmse_errors_per_step)

# Compute averages for per-step quantization errors
avg_baseline_max_errors_per_step = np.mean(all_baseline_max_errors_per_step, axis=0)
avg_baseline_rmse_errors_per_step = np.mean(all_baseline_rmse_errors_per_step, axis=0)
# Extract the prediction time steps (from step 16 onwards)
avg_baseline_max_errors_prediction = avg_baseline_max_errors_per_step[input_time_steps:]
avg_baseline_rmse_errors_prediction = avg_baseline_rmse_errors_per_step[input_time_steps:]
# Compute averages and standard deviations for LLM results
avg_llm_max_diffs_cheb = np.mean(all_llm_max_diffs, axis=0)
avg_llm_rmses_cheb = np.mean(all_llm_rmses, axis=0)
std_llm_max_diffs_cheb = np.std(all_llm_max_diffs, axis=0, ddof=1)
std_llm_rmses_cheb = np.std(all_llm_rmses, axis=0, ddof=1)

def log_ci(mean, std, n, tcrit):
    """
    95% CI for log10 axis
    mean : arithmetic mean of the n samples
    std : sample std of the n samples
    n : number of samples
    tcrit: two-sided t critical value
    """
    se = std / np.sqrt(n)  # SE in linear space
    se_log = se / (mean * np.log(10))  # delta-method SE in log space
    mean_log = np.log10(mean)
    delta_log = tcrit * se_log
    return 10**(mean_log - delta_log), 10**(mean_log + delta_log)

# Calculate confidence intervals
t_critical = stats.t.ppf(0.975, n_ics - 1)  # 95% CI
ci_lower_max_diffs_8B_cheb = []
ci_upper_max_diffs_8B_cheb = []
ci_lower_rmses_8B_cheb = []
ci_upper_rmses_8B_cheb = []

for mean, std in zip(avg_llm_max_diffs_cheb, std_llm_max_diffs_cheb):
    lower, upper = log_ci(mean, std, n_ics, t_critical)
    ci_lower_max_diffs_8B_cheb.append(lower)
    ci_upper_max_diffs_8B_cheb.append(upper)

for mean, std in zip(avg_llm_rmses_cheb, std_llm_rmses_cheb):
    lower, upper = log_ci(mean, std, n_ics, t_critical)
    ci_lower_rmses_8B_cheb.append(lower)
    ci_upper_rmses_8B_cheb.append(upper)

np.savez_compressed(
    "8B_10_step_chebyshev.npz",
    # Averaged LLM metrics on Chebyshev grid
    llm_max_diffs_8B_cheb=avg_llm_max_diffs_cheb,
    llm_rmses_8B_cheb=avg_llm_rmses_cheb,
    std_max_diffs_8B_cheb=std_llm_max_diffs_cheb,
    std_rmses_8B_cheb=std_llm_rmses_cheb,
    # LLM confidence intervals
    ci_lower_max_diffs_8B_cheb=ci_lower_max_diffs_8B_cheb,
    ci_upper_max_diffs_8B_cheb=ci_upper_max_diffs_8B_cheb,
    ci_lower_rmses_8B_cheb=ci_lower_rmses_8B_cheb,
    ci_upper_rmses_8B_cheb=ci_upper_rmses_8B_cheb,
    # Raw results for all initial conditions
    all_llm_max_diffs=all_llm_max_diffs,
    all_llm_rmses=all_llm_rmses,
    # Quant Floor
    avg_baseline_max_errors_prediction=avg_baseline_max_errors_prediction,
    avg_baseline_rmse_errors_prediction=avg_baseline_rmse_errors_prediction,
    # Store the initial conditions and grid info
    stored_initial_conditions=stored_initial_conditions_array,
    use_chebyshev=True,
    Nx=Nx,
    Nt=Nt,
    L=L,
    T=T,
)